# Aditya Sawant's Version of SPN_IP_5Shot.ipynb

### Libraries Used

- tensorflow
- sklearn
- numpy
- matplotlib
- pandas
- scipy
- tensorflow_probability


## Initial setup

### Install Libraries

In [1]:
# %pip install scikit-learn numpy matplotlib scipy tensorflow_probability pandas plotly 
# tensorflow[and-cuda]==2.10 [cuDNN 8.1.1 CUDA 11.2]

### Timer Function

In [1]:
from time import time

def timeIt(func):
    """
    timeIt is a decorator function to time the execution of a function.
    
    :param func: function to be timed
    :return: wrapper function
    """
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__!r} executed in {(t2-t1):.4f}s')
        return result
    return wrap_func

### Plot Data

In [2]:
# trainingData = pd.DataFrame(columns=['Loss', 'Accuracy'])
def plotData(data, testing=False):
    
    if testing:
        accuracy1_values = [item[0] for item in data]
        accuracy2_values = [item[1] for item in data]
        loss1_values = [item[2] for item in data]
        loss2_values = [item[3] for item in data]

        # Create the plot
        plt.figure(figsize=(10, 6))
        plt.plot(loss1_values, color='red', label='Loss')
        plt.plot(accuracy1_values, color='blue', label='Accuracy')
        plt.xlabel('Epoch/Episode')
        plt.ylabel('Value')
        plt.title('Loss and Overall Accuracy 1')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        plt.figure(figsize=(10, 6))
        plt.plot(loss2_values, color='red', label='Loss')
        plt.plot(accuracy2_values, color='blue', label='Accuracy')
        plt.xlabel('Epoch/Episode')
        plt.ylabel('Value')
        plt.title('Loss and Overall Accuracy 2')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        loss_values = [item[0] for item in data]
        accuracy_values = [item[1] for item in data]

        # Create the plot
        plt.figure(figsize=(10, 6))
        plt.plot(loss_values, color='red', label='Loss')
        plt.plot(accuracy_values, color='blue', label='Accuracy')
        plt.xlabel('Epoch/Episode')
        plt.ylabel('Value')
        plt.title('Loss and Accuracy')
        plt.legend()
        plt.grid(True)
        plt.show()
    clear_output(wait=True)

### Import Libraries

In [3]:
import random
import statistics
import os
from IPython.display import clear_output
  
from operator import truediv

import tensorflow as tf
print(tf.version.GIT_VERSION, tf.version.VERSION)
print(tf.test.is_built_with_cuda())
print("Number of GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Number of Devices Available: ", len(tf.config.experimental.list_physical_devices()))
# import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K

from tensorflow.keras import Sequential, layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.compat.v1.distributions import Bernoulli

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from datetime import date, datetime
from tqdm import tqdm

from lib.Data import Data
from lib.Stats import Stats


v2.10.0-rc3-6-g359c3cdfc5f 2.10.0
True
Number of GPUs Available:  1
Number of Devices Available:  2


## Global Variables

In [4]:

# Test each code block individually
TEST_BLOCKS: bool = False
CWD: str = os.getcwd()
VERBOSE: bool = False

SAVE_REPORT: bool = True
SAVE_MODEL: bool = False
# Data Loading and Preprocessing


# Dataset Used : Indian Pines
DATASET: str = 'IP' # IP (indian_pines) PU (pavia_university) SA (salinas) HU (houston) 
PATH_TO_DATASET: str = CWD + '\\Datasets\\'

# PCA
PCA_COMPONENTS: int = 30 # Number of components to keep after PCA reduction

# Window size for forming image cubes
WINDOW_SIZE: int = 11

# Image dimensions after forming image cubes
IMAGE_WIDTH: int
IMAGE_HEIGHT: int
IMAGE_DEPTH: int
IMAGE_CHANNEL: int 
IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_DEPTH, IMAGE_CHANNEL = 11, 11, 30, 1

# Model Parameters

N_TIMES = 1 # Number of times to run the model. Internally, the model is runs each episode N_TIMES times

# Learning Rate
LEARNING_RATE: float = 0.00001

# Temprature Scaling
TAU: float = 1.8

# C (No. of Classes) K (No. of Samples per Class) N (No. of Patches per Class)
TRAIN_C: int = 5 # Number of classes to be used for training
TRAIN_K: int = 5 # Number of patches per class to be used for support during training
TRAIN_N: int = 15 # Number of patches per class to be used for query during training

TUNE_C: int = 3 # Number of classes to be used for testing
TUNE_K: int = 1 # Number of patches per class to be used for support during testing
TUNE_N: int = 4 # Number of patches per class to be used for query during testing

TEST_C: int = 3 # Number of classes to be used for testing
TEST_K: int = 5 # Number of patches per class to be used for support during testing
TEST_N: int = 5 # Number of patches per class to be used for query during testing

# ===================================
# DO NOT REMOVE THIS.
tC = 3   # classes in a test episode 
# Don't know this yet, probably used in the model to calculate loss
MC_LOSS_WEIGHT: int = 5 
# DIRECTLY USED IN PROTOTYPICAL NETWORK CLASS IN TESTING CASE
# ===================================

# Training Epochs
TRAINING_EPOCH: int = 1  # 50

# Training Episode
TRAINING_EPISODE: int = 1 # 100

# Tunning Epochs
TUNNING_EPOCH: int = 41

# Tunning Episode
TUNNING_EPISODE: int = 100

# Testing Epochs
TESTING_EPOCH: int = 1000

# Metrics to be used for evaluation
train_loss = tf.metrics.Mean(name='train_loss')
train_acc = tf.metrics.Mean(name='train_accuracy')
tune_loss = tf.metrics.Mean(name='tune_loss')
tune_acc = tf.metrics.Mean(name='tune_accuracy')
test_loss = tf.metrics.Mean(name='test_loss')
test_acc = tf.metrics.Mean(name='test_accuracy')

trainingData = []
tunningData = []
testingData = []

run_folder =  f'{date.today()}' + '-' + f'{datetime.now().hour}_5_1' + '\\' 

checkpoint_dir = CWD + '\\saves\\' + run_folder + DATASET + '\\' + f'{TRAIN_K}_shot_way' + '\\Train'
checkpoint_prefix_train = os.path.join(checkpoint_dir, "ckpt")

checkpoint_dir1 = CWD + '\\saves\\' + run_folder + DATASET + '\\' + f'{TRAIN_K}_shot_way' + '\\Train\\Tune'
checkpoint_prefix_tune = os.path.join(checkpoint_dir1, "ckpt")

report_path = CWD + f'\\Reports\\Report_{date.today()}_{str(datetime.now()).split(".")[0].split()[1].replace(":", "-")}.txt'
model_save_path = CWD + '\\saves\\' + run_folder + DATASET + '\\' + f'{TRAIN_K}_shot_way' + '\\Train\\encoder.h5'


checkpoint = None  # To be used for loading checkpoints. Declared in the Main Block
ProtoModel = None  # Prototypical Network Object. Declared in the Main Block
model = None  # Model Object. Declared in the Main Block
optimizer = None  # Optimizer Object. Declared in the Main Block

## Model

### Model Construction


In [5]:
def createModel():
    """
    createModel() function creates the model architecture for the 3D CNN model.
    :return: model 
    
    The model architecture is as follows:
    1. Input layer
    2. 3D Convolution layer with 8 filters, kernel size (3,3,7), activation function 'relu' and padding 'same'
    3. Spatial Dropout layer with dropout rate 0.3
    4. 3D Convolution layer with 16 filters, kernel size (3,3,5), activation function 'relu' and padding 'same'
    5. Spatial Dropout layer with dropout rate 0.3
    6. 3D Convolution layer with 32 filters, kernel size (3,3,3), activation function 'relu'
    7. Reshape layer to reshape the output of 3D Convolution layer to 2D
    8. 2D Convolution layer with 64 filters, kernel size (3,3), activation function 'relu'
    9. Flatten layer to flatten the output of 2D Convolution layer
    10. Dropout layer with dropout rate 0.4
    11. Dense layer with 256 neurons and activation function 'relu'
    12. Dropout layer with dropout rate 0.4
    13. Dense layer with 128 neurons and activation function 'relu'
    14. Output layer with 128 neurons and activation function 'relu'
    
    """

    # input_layer = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, IMAGE_CHANNEL))
    
    # output_layer_1_conv = layers.Conv3D(filters=8, kernel_size=(3,3,7), activation='relu',input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, IMAGE_CHANNEL),padding='same')(input_layer)
    
    # output_layer_1_drop3d = layers.SpatialDropout3D(rate=0.3, data_format='channels_last')(output_layer_1_conv,training=True)
    
    # output_layer_2_conv = layers.Conv3D(filters=16, kernel_size=(3,3,5), activation='relu',padding='same')(output_layer_1_drop3d)
    
    # output_layer_2_drop3d = layers.SpatialDropout3D(rate=0.3, data_format='channels_last')(output_layer_2_conv,training=True)
    
    # output_layer_3_conv = layers.Conv3D(filters=32, kernel_size=(3,3,3), activation= 'relu')(output_layer_2_drop3d)
    
    # output_layer_3_reshaped = layers.Reshape((output_layer_3_conv.shape[1], output_layer_3_conv.shape[2], output_layer_3_conv.shape[3]*output_layer_3_conv.shape[4]))(output_layer_3_conv)
    
    # output_layer_4_conv = layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu')(output_layer_3_reshaped)
    
    # output_layer_4_flatten = layers.Flatten()(output_layer_4_conv)
    
    # output_layer_4_drop = layers.Dropout(rate=0.4)(output_layer_4_flatten,training=True)
    
    # output_layer_4_dense = layers.Dense(256, activation='relu')(output_layer_4_drop)
    
    # output_layer_5_conv = layers.Dropout(0.4)(output_layer_4_dense,training=True)
    
    # output_layer_5_dense = layers.Dense(128, activation='relu')(output_layer_5_conv)
    
    # model = Model(inputs=input_layer, outputs=output_layer_5_dense)

    
    input_layer = layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, IMAGE_CHANNEL))

    output_layer_1_conv = layers.Conv3D(filters=8, kernel_size=(3,3,7), activation='relu',input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, IMAGE_CHANNEL),padding='same')(input_layer)

    output_layer_1_drop3d = layers.SpatialDropout3D(rate=0.3, data_format='channels_last')(output_layer_1_conv,training=True)

    output_layer_2_conv = layers.Conv3D(filters=16, kernel_size=(3,3,5), activation='relu',padding='same')(output_layer_1_drop3d)

    output_layer_2_drop3d = layers.SpatialDropout3D(rate=0.3, data_format='channels_last')(output_layer_2_conv,training=True)

    output_layer_3_conv = layers.Conv3D(filters=32, kernel_size=(3,3,3), activation= 'relu')(output_layer_2_drop3d)

    output_layer_3_reshaped = layers.Reshape((output_layer_3_conv.shape[1], output_layer_3_conv.shape[2], output_layer_3_conv.shape[3]*output_layer_3_conv.shape[4]))(output_layer_3_conv)

    output_layer_4_conv = layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu')(output_layer_3_reshaped)

    output_layer_5_SA_dot = layers.Attention()([output_layer_4_conv, output_layer_4_conv])

    output_layer_5_softmax = layers.Softmax()(output_layer_5_SA_dot)

    output_layer_5_SA_concat = layers.Attention(score_mode='concat')([output_layer_5_softmax, output_layer_4_conv])

    output_layer_5_normalization = layers.LayerNormalization()(output_layer_5_SA_concat)

    output_layer_6_flatten = layers.Flatten()(output_layer_5_normalization)

    output_layer_6_drop = layers.Dropout(rate=0.4)(output_layer_6_flatten,training=True)

    output_layer_6_dense = layers.Dense(256, activation='relu')(output_layer_6_drop)

    output_layer_7_conv = layers.Dropout(0.4)(output_layer_6_dense,training=True)

    output_layer_7_dense = layers.Dense(128, activation='relu')(output_layer_7_conv)

    model = Model(inputs=input_layer, outputs=output_layer_7_dense)
    
    print(model.summary())
    return model
    
    
if(TEST_BLOCKS):
    model = createModel()
    print(model.summary())
    

### Prototypical Network

In [6]:
def calc_euclidian_dists(x, y):
    """
    calc_euclidian_dists: Calculates the euclidian distance between two tensors
    :param x: Tensor of shape (n, d)
    :param y: Tensor of shape (m, d)
    :return: Tensor of shape (n, m) with euclidian distances
    """
    n = x.shape[0]
    m = y.shape[0]
    x = tf.tile(tf.expand_dims(x, 1), [1, m, 1])
    y = tf.tile(tf.expand_dims(y, 0), [n, 1, 1])
    return tf.reduce_mean(tf.math.pow(x - y, 2), 2)   

In [11]:
class Prototypical(Model):
    def __init__(self, model, w, h, d, c):
        super(Prototypical, self).__init__()
        self.w, self.h, self.d, self.c = w, h, d, c
        self.encoder = model

    def call(self, support, query, support_labels, query_labels, K, C, N,n_times,training=True):
        # support : support images (25, 11, 11, 30, 1)
        # query : query images (75, 11, 11, 30, 1)
        
        n_class = C                                                               #5
        n_support = K                                                             #5
        n_query = N
        # print('supp',support.shape)                                                               #15 
        # print('que',query.shape)      
        # print('C K N')                                                         #15 
        # print(C, K, N)
        if type(n_times) != int:
            n_times = 0
        if training == True : 
            loss = 0
        mc_predictions = []                                                     # list of predictions for multiple passes
        y = np.zeros((int(C*N),C))
        for i in range(int(C*N)) :
            x = support_labels.index(query_labels[i])
            y[i][x] = 1
            #  basically we are creating OMR sheet for each query image where each column represents the class and each row represents the query image, where the true class is 1 and rest are 0
        for i in range(n_times) :      # n_times passing every query image for calculating variance
            
            cat = tf.concat([support,query], axis=0)       
            # print('cat', cat.shape)                                       # [100, 11, 11, 30, 1]
            z = self.encoder(cat)    
            # print('z', z.shape)                                             # [100, 128]   # build a new computational graph from the provided inputs
            # Divide embedding into support and query
            z_prototypes = tf.reshape(z[:n_class * n_support],[n_class, n_support, z.shape[-1]])   #[5, 5, 128])
            # print('z_p', z_prototypes.shape)   
            # Prototypes are means of n_support examples
            z_prototypes = tf.math.reduce_mean(z_prototypes, axis=1)              #[5, 128]
            # print('z_p mean', z_prototypes.shape)   
            z_query = z[n_class * n_support:]                                     #[75, 128]                         
            # print('z_q', z_query.shape)   
            # Calculate distances between query and prototypes
            dists = calc_euclidian_dists(z_query, z_prototypes)                   #[75, 5]
            # print('dist', dists.shape )   
            # log softmax of calculated distances
            log_p_y = tf.nn.log_softmax(-dists, axis=-1)                          #[75, 5]     this activation function heavily penalizes wrong class prediction as compared to its Softmax counterpart    
            # print('log softmax', log_p_y.shape)   
            loss1 = -tf.reduce_mean((tf.reduce_sum(tf.multiply(y, log_p_y), axis=-1)))   #loss for the current pass                     
            #   print('loss1', loss1)                                                 # []
            loss += loss1                                                         # adding loss for each pass                   
            predictions = tf.nn.softmax(-dists, axis=-1)                         # [75, 5] prediction probability for the search-space classes per query image(for current pass)
            # print('pred softmax', predictions.shape)   
            mc_predictions.append(predictions)           
            # Loop Over (n_times)                                  

        mc_predictions = tf.convert_to_tensor(np.reshape(np.asarray(mc_predictions),(n_times,int(C*N),C)))  #(n_times,75,5)
        # print('all pred', mc_predictions.shape)
        std_predictions = tf.math.reduce_std(mc_predictions,axis=0)                                         # (75,5)
        # print('all std pred', std_predictions.shape)
        std = tf.reduce_sum(tf.reduce_sum(tf.multiply(std_predictions,y),axis=1))
        # print('std', std.shape,std)
        # print('std', std)        
        loss += MC_LOSS_WEIGHT*std
        
        # calculating mean accuracy
        mean_predictions = tf.reduce_mean(mc_predictions,axis=0)                # mean prediction probability for each class (75,5)
        # print('MEAN', mean_predictions.shape)
        mean_eq = tf.cast(tf.equal(                                             # accuracy for the current pass  c
            tf.cast(tf.argmax(mean_predictions, axis=-1), tf.int32),            # check if the index of max probability is equal to the true class index
            tf.cast(tf.argmax(y,axis=-1), tf.int32)), tf.float32)               # argmax returns the index of max probability
        # print('MEAN eq', mean_eq.shape, mean_eq)
        mean_accuracy = tf.reduce_mean(mean_eq)
        # print('MEAN acc', mean_accuracy.shape, mean_accuracy)
        mean_predictions = tf.reduce_mean(mc_predictions,axis=0)                # mean prediction probability for each class (5)
        # print('MEAN preds', mean_predictions.shape, mean_predictions)
        return loss, mean_accuracy, mean_predictions   
      
        if training == False :
            loss = 0
            mc_predictions = []                                                     # list of predictions for multiple passes  
            for i in range(n_times) :                                               # n_times passing the query images for variance calculation
                y = np.zeros((int(C*N),C))                                            # (150,10)
            for i in range(int(C*N)) :
                x = support_labels.index(query_labels[i])                           # creation of 1-hot for the true labels
                y[i][x] = 1  
            # merge support and query to forward through encoder
            cat = tf.concat([support,query], axis=0)                              # [200,9,9,20,1]   
            z = self.encoder(cat)                                                 # [200, 320]
            # Divide embedding into support and query
            z_prototypes = tf.reshape(z[:n_class * n_support],[n_class, n_support, z.shape[-1]])   #[10, 5, 320])
            # Prototypes are means of n_support examples
            z_prototypes = tf.math.reduce_mean(z_prototypes, axis=1)              #[10, 320]
            z_query = z[n_class * n_support:]                                     #[150, 320]                         
            # Calculate distances between query and prototypes
            dists = calc_euclidian_dists(z_query, z_prototypes)                   #[150, 10]
            # log softmax of calculated distances
            log_p_y = tf.nn.log_softmax(-dists, axis=-1)                          #[150, 10]        
            loss1 = -tf.reduce_mean((tf.reduce_sum(tf.multiply(y, log_p_y), axis=-1)))        
            loss += loss1
            predictions = tf.nn.softmax(-dists, axis=-1)                                 # prediction probabilities for the classes for current pass
            mc_predictions.append(predictions)               
          
                                        
        y = np.zeros((int(C*N),C))                                            # (150,10)
        for i in range(int(C*N)) :
            x = support_labels.index(query_labels[i])                           # creation of 1-hot for the true labels
            y[i][x] = 1  
        mean_predictions = tf.reduce_mean(mc_predictions,axis=0)                # mean prediction probability for each class (150,10)
        mean_eq = tf.cast(tf.equal(                                             # accuracy for the current pass
            tf.cast(tf.argmax(mean_predictions, axis=-1), tf.int32), 
            tf.cast(tf.argmax(y,axis=-1), tf.int32)), tf.float32)
        mean_accuracy = tf.reduce_mean(mean_eq)
        mean_pred_index = tf.argmax(mean_predictions,axis=1)
        # mean class-wise accuracies
        mean_correct_class = [[] for i in range(tC)]
        mean_correct_pred = [[] for i in range(tC)]
        classwise_mean_acc = [[] for i in range(tC)]
        for i in range(int(C*N)):
          x = support_labels.index(query_labels[i])
          mean_correct_class[x].append('4')
          if(mean_pred_index[i] == x) :
            mean_correct_pred[x].append('4')
        for i in range(tC) :
           z = len(mean_correct_pred[i])/len(mean_correct_class[i])
           classwise_mean_acc[i].append(z)  
        #std calculation
        std = 0
        for i in range(int(C*N)) :
           x = support_labels.index(query_labels[i])
           p_i = np.array([p[i,:] for p in mc_predictions])
           std_i = tf.math.reduce_std(p_i,axis=0) 
           std_i_true = std_i[x]
           std += std_i_true                                                    # adding std of each class
        # print('std',std)
        loss += MC_LOSS_WEIGHT*std 
        y = np.zeros((int(C*N),C))                                            # (150,10)
        for i in range(int(C*N)) :
            x = support_labels.index(query_labels[i])                           # creation of 1-hot for the true labels
            y[i][x] = 1                                                               
        return loss, mc_predictions, mean_accuracy, classwise_mean_acc, y


        def save(self, model_path):
            self.encoder.save_weights(model_path)

        def load(self, model_path):
            self.encoder(tf.zeros([1, self.w, self.h, self.c]))
            self.encoder.load_weights(model_path)

## Data Loaders for Model

### For Training

In [12]:
def createTrainingEpisode(patches:list, labels:list, K:int, C:int, N:int ):
    """
    createTrainingEpisode creates a training episode for the N-way K-shot learning task.
    
    :param patches: list of all patches classified into different classes.
    :param labels: list of classes from which the traning episode is to be created.
    :param K: number of patches per class in the support set.
    :param C: number of classes in the training episode.
    :param N: number of patches per class in the query set.
    :return queryPatches, queryLabels, supportPatches, supportLabels: training episode
    
    Algorithm:
    - Select N classes from the list of labels. They should be unique.
    - For each class, select K+Q patches. They should be unique.
        - First K patches are support patches.
        - Last Q patches are query patches.
        - Append the support patches to supportPatches.
        - Append the query patches to queryPatches.
        - Append the class label to queryLabels Q times.
    - Shuffle the queryPatches and queryLabels in the same order.
    - Convert the queryPatches and supportPatches to tensors.
    
    """
    
    selectedLabels = random.sample(labels, C)
    supportPatches = []
    supportLabels = list(selectedLabels)
    queryPatches = []
    queryLabels = []
    
    for n in selectedLabels:
        sran_indices = np.random.choice(len(patches[n-1]),K,replace=False)  # for class no X-1: select K samples 
        supportPatches.extend( patches[n-1][sran_indices,:,:,:,:])
        qran_indices = np.random.choice(len(patches[n-1]),N,replace=False)  # N Samples for Query
        queryPatches.extend(patches[n-1][qran_indices,:,:,:,:])
        queryLabels.extend([n]*N)
    
    shuffled = list(zip(queryPatches, queryLabels))
    random.shuffle(shuffled)
    queryPatches, queryLabels = zip(*shuffled)
    
    queryPatches = tf.convert_to_tensor(np.reshape(np.asarray(queryPatches),(C*N,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    supportPatches = tf.convert_to_tensor(np.reshape(np.asarray(supportPatches),(C*K,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    
    return queryPatches, queryLabels, supportPatches, supportLabels


In [13]:
def train_step(support, query, support_labels, query_labels, K, C, N):
    # Forward & update gradients
    with tf.GradientTape() as tape:
        loss, mean_accuracy, mean_predictions = ProtoModel(support, query, support_labels, query_labels, K, C, N,N_TIMES,training=True)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # A gradient simply measures the change in all weights with regard to the change in error. You can also think of a gradient as the slope of a function. The higher the gradient, the steeper the slope and the faster a model can learn. But if the slope is zero, the model stops learning
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Log loss and accuracy for step
    train_loss(loss)
    train_acc(mean_accuracy)


In [14]:
@timeIt
def trainingEpochs(patches, labels, n_epochs, n_episodes):
    """
    trainingEpochs function trains the model for n_epochs and n_episodes.
    
    :param patches: image patches to be trained
    :param labels: corresponding labels to be used
    :param n_epochs: number of epochs
    :param n_episodes: number of episodes
    :return: None
    """
    
    template = 'Epoch {}/{}, Episode {}/{}, Train Loss: {:.2f}, Train Accuracy: {:.2f}'
    
    l=0
    a=0
    # for epoch in tqdm(range(n_epochs), desc='Epochs'):
    #     train_loss.reset_states()
    #     train_acc.reset_states()
    #     for episode in tqdm(range(n_episodes), desc=f'Episodes (Loss: {l:.2f}, Acc: {a:.2f})'):
    
    for epoch in range(n_epochs):
        train_loss.reset_states()
        train_acc.reset_states()
        for episode in range(n_episodes):
            queryPatches, queryLabels, supportPatches, supportLabels = createTrainingEpisode(patches, labels, TRAIN_K, TRAIN_C, TRAIN_N)
            train_step(supportPatches, queryPatches,supportLabels,  queryLabels, TRAIN_K, TRAIN_C, TRAIN_N)
            # clear_output(wait=True)
            l = train_loss.result()*100
            a = train_acc.result()*100
            if(VERBOSE):
                print(template.format(epoch+1, n_epochs, episode+1, n_episodes, train_loss.result()*100, train_acc.result()*100))
                trainingData.append([train_loss.result(),  train_acc.result()*100])
                plotData(trainingData)
        if(epoch and epoch % 5 == 0):
            checkpoint.save(file_prefix=checkpoint_prefix_train)    
        

### For Tuning

In [15]:
def createTunningEpisodes(patches:list, labels:list, K:int, C:int, N:int):
    """
    createTuningEpisodes creates a tuning episode for the N-way K-shot learning task.
    
    :param patches: list of all patches classified into different classes.
    :param labels: list of classes from which the tuning episode is to be created.
    :param K: number of patches per class in the support set.
    :param C: number of classes in the tuning episode.
    :param N: number of patches per class in the query set.
    :return queryPatches, queryLabels, supportPatches, supportLabels: tuning episode
    
    Algorithm:
    - Select C classes from the list of labels. They should be unique.
    - For each selected class.
        - Shuffle the patches of that class.
        - First K patches are support patches.
        - Next N patches are query patches. 
        - Append the support patches to supportPatches.
        - Append the query patches to queryPatches.
        - Append the class label to queryLabels N times.
    - Shuffle the queryPatches and queryLabels in the same order.
    - Convert the queryPatches and supportPatches to tensors.
    
    """

    selected_classes = np.random.choice(labels,C,replace=False)
    supportLabels  = list(selected_classes)
    queryLabels = []
    supportPatches = []
    queryPatches = []
    
    for x in selected_classes :
        y = labels.index(x)
        np.random.shuffle(patches[y])    
        supportPatches.extend(patches[y][:K,:,:,:,:])  # 1st K patches for support set
        queryPatches.extend(patches[y][K:K+N,:,:,:,:])   # next N patches for query set
        queryLabels.extend([x]*N)            
          # next 5 labels for query set
    
    shuffled = list(zip(queryPatches, queryLabels))
    random.shuffle(shuffled)
    queryPatches, queryLabels = zip(*shuffled)
    
    queryPatches = tf.convert_to_tensor(np.reshape(np.asarray(queryPatches),(C*N,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    supportPatches = tf.convert_to_tensor(np.reshape(np.asarray(supportPatches),(C*K,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    
    return queryPatches, queryLabels, supportPatches, supportLabels
    

In [16]:
def tune_step(support, query, support_labels, query_labels, K, C, N):
    # Forward & update gradients
    with tf.GradientTape() as tape:
        loss, mean_accuracy, mean_predictions = ProtoModel(support, query, support_labels, query_labels, K, C, N,N_TIMES,training=True)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Log loss and accuracy for step
    tune_loss(loss)
    tune_acc(mean_accuracy)

In [17]:
@timeIt
def tunningEpochs(patches, labels, n_epochs, n_episodes):
    """
    trainingEpochs function trains the model for n_epochs and n_episodes.
    
    :param patches: image patches to be trained
    :param labels: corresponding labels to be used
    :param n_epochs: number of epochs
    :param n_episodes: number of episodes
    :return: None
    """
    template = 'Epoch {}/{}, Tune Loss: {:.2f}, Tune Accuracy: {:.2f}'
    l=0
    a=0
    for epoch in tqdm(range(n_epochs), desc=f'Epochs (Loss: {l:.2f}, Acc: {a:.2f})'): 
        tune_loss.reset_states()  
        tune_acc.reset_states()    
        for epi in tqdm(range(n_episodes), desc='Episodes'):  
            queryPatches, queryLabels, supportPatches, supportLabels = createTunningEpisodes(patches, labels, TUNE_K, TUNE_C, TUNE_N)    
            tune_step(supportPatches, queryPatches,supportLabels, queryLabels, TUNE_K, TUNE_C, TUNE_N)   
            # clear_output(wait=True)   
        a = tune_acc.result()*100
        l = tune_loss.result()
        if(VERBOSE):
            print(template.format(epoch+1, n_epochs,tune_loss.result(),tune_acc.result()*100))
            tunningData.append([tune_loss.result(),  tune_acc.result()*100])
            plotData(tunningData)
        if (epoch+1)%5 == 0 :
            checkpoint.save(file_prefix = checkpoint_prefix_tune) 

### For Testing

In [47]:
def createTestingEpisode(patches, labels, K, C, i, f):
    selected_classes = labels[i:f]   # [1, 2, 3, 4, 5, 6, 7, 8]
    support_labels = list(selected_classes)
    query_labels = []
    support_patches = []
    query_patches = []
    for x in selected_classes :
        y = labels.index(x)
        support_imgs = patches[y][:K,:,:,:,:]
        query_imgs = patches[y][K:,:,:,:,:]
        support_patches.extend(support_imgs)
        query_patches.extend(query_imgs)
        for i in range(query_imgs.shape[0]) :
            query_labels.append(x)
    print('query_labels', query_labels)
    temp1 = list(zip(query_patches, query_labels)) 
    random.shuffle(temp1) 
    query_patches, query_labels = zip(*temp1)
    x = len(query_labels)
    query_patches = tf.convert_to_tensor(np.reshape(np.asarray(query_patches),(x,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    support_patches = tf.convert_to_tensor(np.reshape(np.asarray(support_patches),(C*K,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    return query_patches, support_patches, query_labels, support_labels,x   

In [19]:
def test_step(support, query, support_labels, query_labels, K, C, y):
    loss, mc_predictions, mean_accuracy, classwise_mean_acc, y = ProtoModel(support, query, support_labels, query_labels, K, C, y,N_TIMES,training=False)
    return loss, mc_predictions, mean_accuracy, classwise_mean_acc, y

In [20]:
@timeIt
def testingEpochs(patches, labels, n_epochs):
    """
    testingEpochs function tests the model for n_epochs.
    
    :param patches: image patches to be trained
    :param labels: corresponding labels to be used
    :param n_epochs: number of epochs
    :return: None
    """
    oa1=0
    oa2=0
    for epoch in tqdm(range(n_epochs), desc=f'Epochs (OA1: {oa1:.2f}, OA2: {oa2:.2f})'):
        test_loss.reset_states()  
        test_acc.reset_states()     
        
        tquery_patches1, tsupport_patches1, query_labels1, support_labels1, x1 = createTestingEpisode(patches,labels,TEST_K,TEST_C,0,3)    
        loss1, mc_predictions1, mean_accuracy1, classwise_mean_acc1, y1 = test_step(tsupport_patches1, tquery_patches1,support_labels1, query_labels1, TEST_K, TEST_C, y=x1/3) 
        tquery_patches2, tsupport_patches2, query_labels2, support_labels2, x2 = createTestingEpisode(patches,labels,TEST_K,TEST_C,3,6)    
        loss2, mc_predictions2, mean_accuracy2, classwise_mean_acc2, y2 = test_step(tsupport_patches2, tquery_patches2,support_labels2, query_labels2, 5, 3, x2/3)
        
        oa1 = mean_accuracy1
        oa2 = mean_accuracy2
        if(VERBOSE):
            print("=========================================")
            print(f"Epoch {epoch+1}/{n_epochs}")
            print("-----------------------------------------")
            print(f"Overall Accuracy 1 (OA1): {mean_accuracy1}")
            # Class Wise Accuracy
            for i in range(TEST_C):
                print(f"Class {i+1} Accuracy: {classwise_mean_acc1[i]}")
            print(f"Loss: {loss1.numpy():.3f}")
            print("-----------------------------------------")
            print(f"Overall Accuracy 2 (OA2): {mean_accuracy2}")
            # Class Wise Accuracy
            for i in range(TEST_C):
                print(f"Class {i+1+TEST_C} Accuracy: {classwise_mean_acc2[i]}")
            print(f"Loss: {loss2.numpy():.3f}")
            print("=========================================")
            
            testingData.append([mean_accuracy1*100, mean_accuracy2*100, loss1.numpy(), loss2.numpy()])
            plotData(testingData, testing=True)
        # else:
            # clear_output(wait=True)
            # print("-----------------------------------------\n" + f"Epoch {epoch+1}/{n_epochs}\n" + "-----------------------------------------\n"
            # + f"Overall Accuracy 1 (OA1): {mean_accuracy1}\n" + f"Overall Accuracy 2 (OA2): {mean_accuracy2}")

        
    return mc_predictions1, mc_predictions2, y1, y2

## Main

In [21]:
# Load Dataset
data = Data(DATASET, PCA_COMPONENTS, WINDOW_SIZE)

indian_pines


In [22]:
X, Y, patches = data.get_data()

In [23]:
NUM_CLASSES, TRAINING_CLASSES, TRAINING_LABELS, TUNNING_LABELS, TESTING_CLASSES, TESTING_LABELS, TRAINING_PATCHES,TUNNING_PATCHES, TESTING_PATCHES = data.load_defaults()

6


In [14]:
# Create instance of the model
model = createModel()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 11, 11, 30,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv3d (Conv3D)                (None, 11, 11, 30,   512         ['input_1[0][0]']                
                                8)                                                                
                                                                                                  
 spatial_dropout3d (SpatialDrop  (None, 11, 11, 30,   0          ['conv3d[0][0]']                 
 out3D)                         8)                                                            

In [24]:
# Create instance of the Prototypical Network
ProtoModel = Prototypical(model, IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_DEPTH, IMAGE_CHANNEL)

In [25]:
# Create instance of the Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

In [26]:
# Create instance of the Checkpoint
checkpoint = tf.train.Checkpoint(optimizer=optimizer, ProtoModel = ProtoModel)

In [55]:
tquery_patches1, tsupport_patches1, query_labels1, support_labels1, x1 = createTestingEpisode(TESTING_PATCHES, TESTING_LABELS, TEST_K,TEST_C,0,3) 

print(x1)

query_labels [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
296


In [35]:
# Train the model
trainingEpochs(patches, TRAINING_LABELS, TRAINING_EPOCH, TRAINING_EPISODE)

supp (25, 11, 11, 30, 1)
que (75, 11, 11, 30, 1)
C K N
5 5 15
cat (100, 11, 11, 30, 1)
z (100, 128)
z_p (5, 5, 128)
z_p mean (5, 128)
z_q (75, 128)
dist (75, 5)
log softmax (75, 5)
pred softmax (75, 5)
all pred (1, 75, 5)
all std pred (75, 5)
std () tf.Tensor(0.0, shape=(), dtype=float32)
MEAN (75, 5)
MEAN eq (75,) tf.Tensor(
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 1. 0.], shape=(75,), dtype=float32)
MEAN acc () tf.Tensor(0.22666667, shape=(), dtype=float32)
MEAN preds (75, 5) tf.Tensor(
[[0.17865865 0.22912592 0.17941977 0.20333011 0.20946552]
 [0.20230205 0.1968834  0.18601921 0.2015391  0.21325625]
 [0.19877985 0.19241264 0.19219927 0.2146082  0.20200007]
 [0.16858739 0.2201357  0.20168291 0.2085933  0.20100075]
 [0.1856264  0.22867882 0.1959179  0.2065782  0.18319872]
 [0.22029643 0.18088785 0.17235488

In [29]:
tunningEpochs(TUNNING_PATCHES, TESTING_LABELS, TUNNING_EPOCH, TUNNING_EPISODE)

Epoch 41/41, Tune Loss: 0.11, Tune Accuracy: 100.00
Function 'tunningEpochs' executed in 457.1728s


In [30]:
mc_predictions1, mc_predictions2, y1, y2 =  testingEpochs(TESTING_PATCHES, TESTING_LABELS, TESTING_EPOCH)

-----------------------------------------
Epoch 1000/1000
-----------------------------------------
Overall Accuracy 1 (OA1): 0.9459459185600281
Overall Accuracy 2 (OA2): 0.9405940771102905
Function 'testingEpochs' executed in 1439.1789s


In [31]:

from lib.Stats import Stats
stats = Stats(mc_predictions1, mc_predictions2, y1, y2)

In [32]:
stats.printReport()

92.21102068290294 Kappa accuracy (%)


94.32387312186978 Overall accuracy (%)


97.35057471264366 Average accuracy (%)




              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       1.00      0.93      0.96       232
           2       0.59      1.00      0.74        23
           3       0.45      1.00      0.62        15
           4       1.00      0.91      0.95       200
           5       1.00      1.00      1.00        88

    accuracy                           0.94       599
   macro avg       0.84      0.97      0.88       599
weighted avg       0.97      0.94      0.95       599



[[ 41   0   0   0   0   0]
 [  0 216  16   0   0   0]
 [  0   0  23   0   0   0]
 [  0   0   0  15   0   0]
 [  0   0   0  18 182   0]
 [  0   0   0   0   0  88]]


In [33]:
if(SAVE_REPORT):
    stats.saveReport(report_path)

In [34]:
if(SAVE_MODEL):
    ProtoModel.save(model_save_path)